Aquí vamos a trabajar con los datos de nuestro dataset de felidad, pero le vamos a hacer algunas modificaciones: Le vamos a introducir ciertos datos corruptos que nuestro regresor va a tener que lidiar.

In [21]:
import pandas as pd
from sklearn.linear_model import (RANSACRegressor, HuberRegressor)  #Podemos hacerlo de esta manera, ya que ambos pertenecen al mismo módulo)

from sklearn.svm import SVR #Máquinas de soporte Vectorial

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error #La métrica que usaremos

In [22]:
dataset = pd.read_csv('Datasets/felicidad.csv')
dataset.head()

,country,rank,score,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [23]:
X= dataset.drop(['country', 'score'], axis=1)
Y= dataset['score']
X.head()


,rank,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
0,1,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,2,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,3,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,4,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,5,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=42)


### DATO:

Una manera professional de trabajar con varios estimadores al mismo tiempo, de tal manera que podamos hacerlo sin escribir muchas líneas de código.

In [25]:
#Usaremos un diccionario
#Nota: RANSA no es un estimador. Es un meta-estimador
estimadores = { 'SVR':SVR(gamma= 'auto', C=1.0, epsilon = 0.1),
                'RANSAC': RANSACRegressor(),    #Podemos trabajar con diferentes estimadores como parámetro en RANSAC, ya que es un "MetaEstimador".RANSAC trabaja con la regresión lineal por defecto.
                'Huber': HuberRegressor(epsilon = 1.35)
    
}

In [28]:
for name, estimador in estimadores.items():
    estimador.fit(X_train, Y_train)
    predictions = estimador.predict(X_test)
    
    print(name)
    
    print("MSE: ", mean_squared_error(Y_test, predictions))
    print("Score", estimador.score(X_test, Y_test))

SVR
MSE:  0.024636332906412182
Score 0.9780019790127239
RANSAC
MSE:  1.1899164154047088e-19
Score 1.0
Huber
MSE:  1.4911035802611992e-06
Score 0.9999986685791276


C:\Users\Usuario\anaconda3\envs\Platzi- Enviroment\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [27]:
#Esto es solo para que le eches un vistaso a lo que obtenemos con items
estimadores.items()

dict_items([('SVR', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)), ('RANSAC', RANSACRegressor(base_estimator=None, is_data_valid=None, is_model_valid=None,
                loss='absolute_loss', max_skips=inf, max_trials=100,
                min_samples=None, random_state=None, residual_threshold=None,
                stop_n_inliers=inf, stop_probability=0.99, stop_score=inf)), ('Huber', HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False))])

### Resultados

Como podemos observar, el estimador con menor error es RANSAC.Por otrolado, el estimador con mayor error es el SVR. Con ello demostramos que los Meta-estimadores, al suprimir o penalizar a los atípicos, logran reducir el error de manera significativa. 